# Variables definition

In [1]:
import os
from dotenv import load_dotenv # requires python-dotenv

load_dotenv("./../../config/credentials_my.env")

openai_endpoint       = os.environ["azure_openai_endpoint"]
openai_api_key        = os.environ["azure_openai_api_key"]
openai_api_version    = "2024-12-01-preview" # or "2024-10-01-preview"
azure_deployment_name = "o1" # "gpt-4o-2024-08-06" 

messages = [
    {
      "role": "user",
      "content": ""
    }
]

with open("openai_caching_request01.txt", "r") as file:
    request01 = file.read()

with open("openai_caching_request02.txt", "r") as file:
    request02 = file.read()

print(f"openai_endpoint: {openai_endpoint}")

openai_endpoint: https://mmoaiswc-01.openai.azure.com/


# Invoking Open AI via HTTP
Invoking Azure OpenAI through HTTP requires to append `openai/` to the base Azure OpenAI endpoint, e.g.
- [https://mmoaiscus-01.openai.azure.com/](https://mmoaiscus-01.openai.azure.com/) must become:
- [https://mmoaiscus-01.openai.azure.com/openai/](https://mmoaiscus-01.openai.azure.com/openai)

In [2]:
import requests

url = f"{openai_endpoint}openai/deployments/{azure_deployment_name}/chat/completions?api-version={openai_api_version}" # "openai/" was appended
print(f'This is the Open AI endpoint and body that will be invoked:\n- {url}')

This is the Open AI endpoint and body that will be invoked:
- https://mmoaiswc-01.openai.azure.com/openai/deployments/o1/chat/completions?api-version=2024-12-01-preview


In [3]:
messages[0]["content"] =  request01 # switch between request01 and request02, and run it again
messages_json = {"messages": messages}

response_http = requests.post(url, headers = {'api-key':openai_api_key}, json = {"messages": messages})

response_json = response_http.json()

if (int(response_http.status_code / 100)) != 2:
    print(f"Error: {response_http.text}")

response_json.get("usage")

{'completion_tokens': 1695,
 'completion_tokens_details': {'accepted_prediction_tokens': 0,
  'audio_tokens': 0,
  'reasoning_tokens': 256,
  'rejected_prediction_tokens': 0},
 'prompt_tokens': 1044,
 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0},
 'total_tokens': 2739}

# Invoking Open AI via Azure SDK
In this case, the SDK expects the "pure" Azure OpenAI endpoint [https://mmoaiscus-01.openai.azure.com/](https://mmoaiscus-01.openai.azure.com/), which will be used internally to make the call.<br/>
In other words, in a trasparent way for the Developer whose task is just to pass the "pure" Azure OpenAI endpoint, the Azure OpenAI SDK receives it and then internally appends the `openai/` string to the base endpoint to make it work.

In [4]:
from openai import AzureOpenAI

client = AzureOpenAI(
    azure_endpoint = openai_endpoint,
    api_key        = openai_api_key,
    api_version    = openai_api_version
)

In [5]:
import json


In [6]:
messages[0]["content"] =  request01 # switch between request01 and request02, and run it again
messages_json = {"messages": messages}

response_sdk = client.chat.completions.create(
    model    = azure_deployment_name,
    messages = messages)

json.loads(response_sdk.model_dump_json())["usage"]

{'completion_tokens': 2063,
 'prompt_tokens': 1044,
 'total_tokens': 3107,
 'completion_tokens_details': {'accepted_prediction_tokens': 0,
  'audio_tokens': 0,
  'reasoning_tokens': 704,
  'rejected_prediction_tokens': 0},
 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}